### Final EWMACD Method

In [1]:
# globals (dev)
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
GRP_LYR_FILE = r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx"

# set gdal global environ
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'
os.environ['CPL_VSIL_CURL_ALLOWED_EXTENSIONS '] = 'tif'
os.environ['VSI_CACHE '] = 'TRUE'
os.environ['GDAL_HTTP_MULTIRANGE '] = 'YES'
os.environ['GDAL_HTTP_MERGE_CONSECUTIVE_RANGES '] = 'YES'

# also set rasterio env variables
rasterio_env = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS':'tif',
    'VSI_CACHE': True,
    'GDAL_HTTP_MULTIRANGE': 'YES',
    'GDAL_HTTP_MERGE_CONSECUTIVE_RANGES': 'YES'
}

# safe imports
import sys                      # arcgis comes with these
import shutil                   # arcgis comes with these
import datetime                 # arcgis comes with these
import numpy as np              # arcgis comes with these
import pandas as pd             # arcgis comes with these
import arcpy                    # arcgis comes with these
import tempfile                 # arcgis comes with these
#import matplotlib.pyplot as plt
import smtplib
import mimetypes
from datetime import datetime   # arcgis comes with these
from email.message import EmailMessage
from email.utils import make_msgid



# risky imports (not native to arcgis)
try:
    import xarray as xr
    import dask
    import rasterio
    import pystac_client
    import osr
    import json
    from scipy.signal import savgol_filter
    from odc import stac
    from osgeo import gdal
    from osgeo import ogr
    from osgeo import osr
except:
    arcpy.AddError('Python libraries xarray, dask, rasterio, pystac, or odc not installed.')
    raise # return

# import tools
try:
    # shared folder
    sys.path.append(FOLDER_SHARED)
    import arc, satfetcher, tools

    # module folder
    sys.path.append(FOLDER_MODULES)
    import nrt, cog_odc, cog
except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise
    
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=dask.array.core.PerformanceWarning)
    
# grab parameter values 
in_feat = r"C:\Users\Lewis\Desktop\nrt_projects\test2\monitoring_areas.gdb\monitoring_areas"
in_ongoing = False
in_time_interval = 24
#in_email_from = parameters[3].value                # email from 
#in_smtp_server = parameters[4].value               # email smtp server 
#in_smtp_port = parameters[5].value                 # email smtp port 
#in_smtp_username = parameters[6].value             # email smtp username 
#in_smtp_password = parameters[7].value             # email smtp password 


# # # # #
# notify user and set up progress bar
#arcpy.AddMessage('Beginning NRT Monitoring of areas.')
#arcpy.SetProgressor(type='step', 
                    #message='Preparing parameters...',
                    #min_range=0, max_range=20)
        
# set up initial continous monitoring var
continue_monitoring = True

# check if time interval is > 0
#in_time_interval = in_time_interval * 60 * 60
#if in_time_interval <= 0:
    #arcpy.AddError('Time interval must be above 0 hours.')
    #raise


# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Preparing parameters...')
#arcpy.SetProgressorPosition(1)

# get path to monitoring areas feature
feat_desc = arcpy.Describe(in_feat)
in_feat = os.path.join(feat_desc.path, feat_desc.name)


# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Validating monitoring areas...')
#arcpy.SetProgressorPosition(2)

# validate monitoring area feature class
try:
    nrt.validate_monitoring_areas(in_feat)
except:
    arcpy.AddError('Monitoring areas feature is invalid.')
    raise # return

    
# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Loading monitoring area json data...')
#arcpy.SetProgressorPosition(2)
    
# prepare path to expected json file
in_path = os.path.dirname(in_feat)
in_path = os.path.splitext(in_path)[0]
in_path = os.path.dirname(in_path)
#in_data_path = os.path.join(in_path, 'data.json')
    
    
# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Loading monitoring area features...')
#arcpy.SetProgressorPosition(2)

# set required fields
fields = [
    'area_id', 
    'platform', 
    's_year', 
    'e_year', 
    'index', 
    'persistence', 
    'rule_1_min_conseqs', 
    'rule_1_inc_plateaus', 
    'rule_2_min_zone', 
    'rule_3_num_zones', 
    'ruleset', 
    'alert', 
    'method',
    'alert_direction', 
    'email', 
    'ignore', 
    'color_border',
    'color_fill',
    'global_id', 
    'SHAPE@'
]

# get feature count and data
try:
    #feats = arcpy.da.SearchCursor(in_feat, fields)
    feats = []
    with arcpy.da.SearchCursor(in_feat, fields) as cursor:
        for row in cursor:
            feats.append(row)
except:
    arcpy.AddError('Could not open monitoring areas feature.')
    raise # return

OSError: "C:\Users\Lewis\Desktop\nrt_projects\test2\monitoring_areas.gdb\monitoring_areas" does not exist

In [2]:
from importlib import reload
reload(nrt)

<module 'nrt' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\modules\\nrt.py'>

In [84]:
# todo:
# remove field rule 1 inc plateaus ... ?
# remove field rule 2 bidirection. we do this via alert dir now - DONE
# change rule 2 min stdv to min zone - DONE
# consider a 'negative remover in positive areas' and vice versa. consider during alert?... add as extra alert dirs? - DONE
# append all mon area field info to netcdf attr, check at start of run for change, delete cube if change
# check if ruleset contains rule without a value entered during area creation (fields accepts nulls) - FORCED REQUIRED!

In [85]:
# 'inc_any':      'Incline only (any)', 
# 'dec_any':      'Decline only (any)', 
# 'inc_pos':      'Incline only (+ zones only)', 
# 'dec_neg':      'Decline only (- zones only)', 
# 'both_any':     'Incline or Decline (any)',
# 'both_pos_neg': 'Incline or Decline (+/- zones only)',

In [86]:
# tips
# use higher persistence for dynamic (1) and lower for static (0.5)
# use decline in - or pos in + only for dynamic to avoid new regime shifts triggering alarm
# turn off spikes for dynamic

In [1]:
import arcpy

In [2]:
in_new_feat = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\project\tenement-tools-project\tenement-tools-project.gdb\Polygons_45'

In [4]:
poly_wgs = arcpy.management.Project(in_dataset=in_new_feat, 
                                    out_dataset='poly_prj', 
                                    out_coor_system=4326)

cursor = arcpy.da.SearchCursor(poly_wgs, ['SHAPE@WKB'])
poly = cursor.next()

In [6]:
poly[0]

bytearray(b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x17\x00\x00\x00\xc8\xcc\x96\x90\xb5\x11]@8\xa9\x96w\x8cU@\xc0@\xbb\xfc!\xb2\x11]@X(\xc8\x1e\x8dU@\xc0\xa0\xd1\xa7l\xb0\x11]@x\xc9\xdf \x8eU@\xc0(\x03\x1f\x07\xae\x11]@\x80\x95\x9b\x8c\x90U@\xc0p\xef\xf0*\xad\x11]@\xc0V$\xb4\x92U@\xc0\x90s\x0b\xc5\xac\x11]@\xb8\xe5\xe7\x0f\x98U@\xc0hr\x9a\xa0\xae\x11]@\x10\x82\xdaw\x9eU@\xc0\x98\xe7\xa7\xf6\xb1\x11]@`:\x94\xd1\xa3U@\xc0\xe0\xfe\x85\xb3\xb5\x11]@\xb8S\x86y\xa8U@\xc0pk\x1a\xfa\xb8\x11]@\x98y;\xed\xa9U@\xc0 \xc1\x01X\xbd\x11]@82\x84\xd3\xaaU@\xc0\xb0\xb4\x9a\xb5\xc1\x11]@8\x8e\xcd+\xaaU@\xc0x\xbe\x07\xfe\xc3\x11]@\x98W\xfeG\xa9U@\xc0Hn~\xaa\xc7\x11]@\xb8\xc8\xed_\xa5U@\xc0\x90\xb0\xc0\xdb\xcc\x11]@\x08\x8d\xab\xba\x9cU@\xc0H\xf2\xe1\xce\xcf\x11]@\xf0\xe4AJ\x91U@\xc0\x80\x9dl\xb4\xd0\x11]@\xa0\x90\xc6\xc9\x86U@\xc0x\xaa\xd5\x87\xd0\x11]@\x10\xdf\xb2k\x84U@\xc0\xd0\xbb\x92\x00\xcf\x11]@8>\xf2\xb8|U@\xc0\x80\xee\xde3\xcd\x11]@\xa0\xc6\t\x1bwU@\xc0\x08@\x91\xfe\xc9\x11]@\xc0Ph\x08pU@\xc0\xc8\xb1